In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate
from langchain.schema.agent import AgentFinish
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from pydantic import BaseModel, Field
from langchain.prompts import MessagesPlaceholder
from typing import List, Optional
import pandas as pd

In [14]:
class movieDetails(BaseModel):
    movie_genre :List[str] = Field(description="The genres of the movies/movie you like or you have watched before seperated by comma")
    movies_watched: Optional[List[str]] = Field(description="The names of the movies you have watched before or like, separated by comma")


In [ ]:
@tool(args_schema=movieDetails)
def get_movies_you_might_love(movieDetails: movieDetails) -> dict:
    """Recommends movies based on genres or previously watched movies."""

    movie_df = pd.read_csv("./imdb_top_1000.csv")

    if movieDetails.movie_genre and len(movieDetails.movie_genre) > 0:
        genres_you_like = [genre.strip().lower() for genre in movieDetails.movie_genre] 
        genre_filter = movie_df['Genre'].str.contains('|'.join(genres_you_like), case=False, na=False)
        movies_with_same_genre = movie_df[genre_filter]
        return {"movies you might like": list(movies_with_same_genre['Series_Title'])}
    elif movieDetails.movies_watched and len(movieDetails.movies_watched) > 0:
        movies_watched = [movie.strip().lower() for movie in movieDetails.movies_watched]  
        movie_filter = movie_df['Series_Title'].str.contains('|'.join(movies_watched), case=False, na=False)
        movies_watched_data = movie_df[movie_filter]
       
        genre_of_watched_movies = set(movies_watched_data['Genre'].str.split(',').explode().str.strip().unique())
        genre_filter = movie_df['Genre'].str.contains('|'.join(genre_of_watched_movies), case=False, na=False)
        movies_with_similar_genres = movie_df[genre_filter]
        return {"movies you might like": list(movies_with_similar_genres['Series_Title'])}

    else:
        return {"MovieError": "Please provide either a genre or a list of movies you've watched."}